# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [1]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read Data

In [4]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/v2(gender_code_change)_updated_toxicity.csv')
data

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,চোদনার মেয়া বদনার নাতনি । । সালী জাত ফকির । ...,[],1,1,1,0,চোদনার পুত বদনার নাতি।। <Gender> জাত ফকিন্নি।।...,চোদনার পুত বদনার নাতি।। <Gender> জাত ফকিন্নি।।...
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,[],1,1,1,0,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",[],1,1,1,0,"তোদের মত সবাই হাবলা <Gender> না , তোদের কি কোন...","তোদের মত সবাই হাবলা <Gender> না , তোদের কি কোন..."
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",[],1,1,1,0,"তোদের মত সবাই হাবলা <Gender> না , তোদের কি কোন...","তোদের মত সবাই হাবলা <Gender> না , তোদের কি কোন..."
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,যদি বদরুল শিব িরের নেত্রী হয় তাহলে ফাসি দেওয়া...,['বদরুল'],1,1,1,0,যদি বদরুল শিবিরের <Gender> হয় তাহলে ফাসি দেওয়া...,যদি <Name> শিবিরের <Gender> হয় তাহলে ফাসি দেওয়...
...,...,...,...,...,...,...,...,...,...
7051,নিশো ভাই তুমি সেরাদারুন নাটক,অমৃতা আফা তুমি সেরাদারুন নাটক,['নিশো'],0,0,0,0,নিশো <Gender> তুমি সেরাদারুন নাটক,<Name> <Gender> তুমি সেরাদারুন নাটক
7052,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,['নিশো'],0,0,1,1,নিশো <Gender> তুমি সেরাদারুন নাটক,<Name> <Gender> তুমি সেরাদারুন নাটক
7053,নিশো ভাই তুমি সেরাদারুন নাটক,দুর্গা ভাবী তুমি সেরাদারুন নাটক,['নিশো'],0,0,1,1,নিশো <Gender> তুমি সেরাদারুন নাটক,<Name> <Gender> তুমি সেরাদারুন নাটক
7054,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,[],0,0,0,0,সত্যি বলতে কি এই রকম <Gender> টাইপের ভালোবাসা ...,সত্যি বলতে কি এই রকম <Gender> টাইপের ভালোবাসা ...


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [5]:
temp = data[data['Predict Biasness']!= 1]
test = data[data['Predict Biasness']== 1]

In [6]:
t1 = temp[['Original Sentence', 'original_lebel']]
t2 = temp[['Converted Sentence', 'original_lebel']]
t2.columns = ['Original Sentence', 'original_lebel']
temp = pd.concat([t1, t2], axis = 0)
temp

,Original Sentence,original_lebel
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,1
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...",1
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...",1
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,1
...,...,...
7049,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে মেয়ে কলেজ...,0
7050,অসম্ভব সুন্দর এত ভালো লেগেছে নাটকটা অনেকদিন পর...,0
7051,অমৃতা আফা তুমি সেরাদারুন নাটক,0
7054,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,0


In [7]:
validation = temp.groupby('original_lebel', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

In [8]:
train

,Original Sentence,original_lebel
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,1
8,চুদির ভাই বাংলায় লিখতে পারো নাই,1
9,চুদির ভাই বাংলায় লিখতে পারো নাই,1
10,"যখন কেউ বলে অস্ত্রের চাইতে কলম বড়, তখন অস্ত্রধ...",1
...,...,...
7048,অসাধারণ ভাল লাগল ড্রামাটা ধন্যবাদ পরিচালিকা ও ...,0
7049,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে মেয়ে কলেজ...,0
7050,অসম্ভব সুন্দর এত ভালো লেগেছে নাটকটা অনেকদিন পর...,0
7054,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,0


In [9]:
validation

,Original Sentence,original_lebel
4952,অসাধারণ একটি নাটক নুজহাত ভাবী,0
3756,নাটক টা দেখে চোখের পানি ধরে রাখতে পারলাম নাএকট...,0
5923,অসাধারণ নাটক ধন্যবাদ মাইশা ভাবী কে এবং আমার ...,0
6542,আপু নাটকটা খুব খুব সুন্দর লাগলো,0
6090,অপুর্ব ভাইয়ার নাটক আমার সবসময় ভাল লাগেতার অভিন...,0
...,...,...
3301,"দোস্ত এইটাই বুঝলাম,কেয়ামতের দিন ফেরেশতাঃ কিরে ...",1
743,তর বাপেরে চুদি তর মায়ের কান্দে ফালাইয়া,1
2378,কোথায় গেল খানকির পোলা মুরগী কবির। শুয়োরের বাচ্...,1
2359,ও মাগো এই রাতের বেলাই এত বড় জরুরি খবর টা দেওয়া...,1


In [10]:
test

,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked
25,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য র...,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য র...,[],1,1,0,1,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য <...,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য <...
30,আমার মনে হয় হিরো আলম কে একটা চবি বানানো দরকার ...,আমার মনে হয় ইমদাদুল কে একটা চবি বানানো দরকার...,"['হিরো আলম', 'হিরো আলমের', 'জাহিদ']",1,0,1,1,আমার মনে হয় <Gender> আলম কে একটা চবি বানানো দর...,আমার মনে হয় <Gender> <Name> কে একটা চবি বানানো...
31,আমার মনে হয় হিরো আলম কে একটা চবি বানানো দরকার ...,আমার মনে হয় অর্ক কে একটা চবি বানানো দরকার কা...,"['হিরো আলম', 'হিরো আলমের', 'জাহিদ']",1,0,1,1,আমার মনে হয় <Gender> আলম কে একটা চবি বানানো দর...,আমার মনে হয় <Gender> <Name> কে একটা চবি বানানো...
68,বি চৌধুরীর নাম নিশানা ডুবিয়ে দিতে মা চৌধুরী যথ...,বি অর্ক র নাম নিশানা ডুবিয়ে দিতে বাবা অর্ক ...,"['বি চৌধুরীর', 'চৌধুরী']",1,1,0,1,বি চৌধুরীর নাম নিশানা ডুবিয়ে দিতে <Gender> <Ge...,<Name> নাম নিশানা ডুবিয়ে দিতে <Gender> <Gende...
88,মাদক আর মাগি দেশ টা কে নষ্ট করে দিল ৷,মাদক আর মাইগ্যা দেশ টা কে নষ্ট করে দিল ৷,[],1,1,0,1,মাদক আর <Gender> দেশ টা কে নষ্ট করে দিল ৷,মাদক আর <Gender> দেশ টা কে নষ্ট করে দিল ৷
...,...,...,...,...,...,...,...,...,...
7000,আলহামদুলিল্লাহ নেত্রী বেগম খালেদা জিয়ার সুস্থ...,আলহামদুলিল্লাহ নেতা গুলশান সুস্থতা এবং দীর্ঘা...,[' বেগম খালেদা জিয়ার'],0,1,0,1,আলহামদুলিল্লাহ <Gender> <Gender> খালেদা জিয়ার...,আলহামদুলিল্লাহ <Gender> <Gender> <Name> সুস্থ...
7019,অপুর্ব ভাই আপনার সাথে দেখা করব আমার নাম নাঈম,দুর্গা বোন আপনার সাথে দেখা করব আমার নাম নাঈম,"['অপুর্ব ','নাঈম']",0,0,1,1,অপুর্ব <Gender> আপনার সাথে দেখা করব আমার নাম নাঈম,<Name> <Gender> আপনার সাথে দেখা করব আমার নাম <...
7041,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,['তাহসান'],0,0,1,1,নায়ক হিসেবে তাহসান <Gender> থাকলে ভালো হতে,নায়ক হিসেবে <Name> <Gender> থাকলে ভালো হতে
7052,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,['নিশো'],0,0,1,1,নিশো <Gender> তুমি সেরাদারুন নাটক,<Name> <Gender> তুমি সেরাদারুন নাটক


# Finetune with ner + original all data

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model

In [13]:
model_path = "FredZhang7/one-for-all-toxicity-v3"
tokenizer, b_model = load_transformer_based_model(model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at FredZhang7/one-for-all-toxicity-v3 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

Total Pretrained tokens:  119547
Total number of tokens after adding the new ones:  119549


In [15]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, max_length=200 )
dataset_val= BertDataset( validation, tokenizer, max_length=200)
dataset_test= BertDataset(test, tokenizer, max_length=200)
train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle=False)


In [ ]:
id,am, tid, l = next(iter(train_loader))
x = b_model(id, am,tid)
x.hidden_states[0].shape

torch.Size([16, 200, 768])

In [16]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [ ]:
id,am, tid, l = next(iter(train_loader))
x,_ = masked_model(id, am,tid)
x.shape

torch.Size([16, 1])

In [17]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


In [18]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = '')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'/content/drive/MyDrive/#Research/# GB/saved_model/baseline_toxicity.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.030117,0.761262,0.019271,0.883614,239.879560,239.881605,0.0001,16,cuda
1,1,2,0.015106,0.906649,0.013536,0.912711,246.957330,505.137306,0.0001,16,cuda
2,1,3,0.010743,0.937485,0.011005,0.933767,248.466619,770.311655,0.0001,16,cuda
3,1,4,0.008358,0.952180,0.009992,0.936830,247.859713,1035.511492,0.0001,16,cuda
4,1,5,0.006719,0.963623,0.009156,0.945253,248.413601,1300.975911,0.0001,16,cuda
5,1,6,0.005525,0.971573,0.008942,0.945636,248.346135,1567.132071,0.0001,16,cuda
6,1,7,0.004731,0.976391,0.008493,0.950230,248.302107,1832.456055,0.0001,16,cuda
7,1,8,0.003982,0.980607,0.008233,0.952144,248.410310,2098.043067,0.0001,16,cuda
8,1,9,0.003430,0.984341,0.008103,0.952910,248.716130,2363.798113,0.0001,16,cuda
9,1,10,0.003001,0.984582,0.008036,0.955207,248.712253,2629.579446,0.0001,16,cuda


Saved model and optimizer at /content/drive/MyDrive/#Research/# GB/saved_model/baseline_toxicity.pth


In [ ]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/content/drive/MyDrive/#Research/# GB/saved_model/baseline_toxicity.pth')

Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/Senti_Masked.pth


In [19]:

class BertDataset(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,0] #0th column all data

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])

def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()
dataset_test= BertDataset(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 33/33 [00:10<00:00,  3.14it/s]


In [20]:
test['pred'] = [x[0] for pred in p for x in pred.tolist()]
test

<ipython-input-20-4aadd1bb9061>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred'] = [x[0] for pred in p for x in pred.tolist()]


,Original Sentence,Converted Sentence,Name,original_lebel,Predicted Value for Original Sentence,Predicted Value for ner converted,Predict Biasness,Gender Masked,Gender and Name Masked,pred
25,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য র...,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য র...,[],1,1,0,1,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য <...,ইংল্যান্ড আর বাংলাদেশ দুইজন একটা ১ জনের জন্য <...,1
30,আমার মনে হয় হিরো আলম কে একটা চবি বানানো দরকার ...,আমার মনে হয় ইমদাদুল কে একটা চবি বানানো দরকার...,"['হিরো আলম', 'হিরো আলমের', 'জাহিদ']",1,0,1,1,আমার মনে হয় <Gender> আলম কে একটা চবি বানানো দর...,আমার মনে হয় <Gender> <Name> কে একটা চবি বানানো...,1
31,আমার মনে হয় হিরো আলম কে একটা চবি বানানো দরকার ...,আমার মনে হয় অর্ক কে একটা চবি বানানো দরকার কা...,"['হিরো আলম', 'হিরো আলমের', 'জাহিদ']",1,0,1,1,আমার মনে হয় <Gender> আলম কে একটা চবি বানানো দর...,আমার মনে হয় <Gender> <Name> কে একটা চবি বানানো...,1
68,বি চৌধুরীর নাম নিশানা ডুবিয়ে দিতে মা চৌধুরী যথ...,বি অর্ক র নাম নিশানা ডুবিয়ে দিতে বাবা অর্ক ...,"['বি চৌধুরীর', 'চৌধুরী']",1,1,0,1,বি চৌধুরীর নাম নিশানা ডুবিয়ে দিতে <Gender> <Ge...,<Name> নাম নিশানা ডুবিয়ে দিতে <Gender> <Gende...,1
88,মাদক আর মাগি দেশ টা কে নষ্ট করে দিল ৷,মাদক আর মাইগ্যা দেশ টা কে নষ্ট করে দিল ৷,[],1,1,0,1,মাদক আর <Gender> দেশ টা কে নষ্ট করে দিল ৷,মাদক আর <Gender> দেশ টা কে নষ্ট করে দিল ৷,1
...,...,...,...,...,...,...,...,...,...,...
7000,আলহামদুলিল্লাহ নেত্রী বেগম খালেদা জিয়ার সুস্থ...,আলহামদুলিল্লাহ নেতা গুলশান সুস্থতা এবং দীর্ঘা...,[' বেগম খালেদা জিয়ার'],0,1,0,1,আলহামদুলিল্লাহ <Gender> <Gender> খালেদা জিয়ার...,আলহামদুলিল্লাহ <Gender> <Gender> <Name> সুস্থ...,1
7019,অপুর্ব ভাই আপনার সাথে দেখা করব আমার নাম নাঈম,দুর্গা বোন আপনার সাথে দেখা করব আমার নাম নাঈম,"['অপুর্ব ','নাঈম']",0,0,1,1,অপুর্ব <Gender> আপনার সাথে দেখা করব আমার নাম নাঈম,<Name> <Gender> আপনার সাথে দেখা করব আমার নাম <...,0
7041,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,['তাহসান'],0,0,1,1,নায়ক হিসেবে তাহসান <Gender> থাকলে ভালো হতে,নায়ক হিসেবে <Name> <Gender> থাকলে ভালো হতে,0
7052,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,['নিশো'],0,0,1,1,নিশো <Gender> তুমি সেরাদারুন নাটক,<Name> <Gender> তুমি সেরাদারুন নাটক,0


Result

In [21]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  528 Total mismatch:  73 Accuracy:  0.8617424242424243


In [22]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

528


# Prediction of NER Converted text data

In [23]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])
dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 33/33 [00:10<00:00,  3.01it/s]


Total test data:  528 Total mismatch:  82 Accuracy:  0.8446969696969697 Bias:  25


<ipython-input-23-8451d832cb32>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]


In [24]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  528 Total mismatch:  82 Accuracy:  0.8446969696969697 Bias:  25


In [25]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_toxicity_reSult.csv', index = False)